In [31]:
# installing the libraries for transformers
#!pip install -U -q sentence-transformers transformers bitsandbytes accelerate sentencepiece

# install libarary to convert latex to text
#!pip install -U pylatexenc


In [32]:
import logging 
logging.basicConfig(level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S')
logging.info("starting script")

2024-11-17 20:16:36 - INFO - starting script


In [33]:
""" # test tokenize latex
from transformers import AlbertTokenizer
import torch

# Load the tokenizer
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

# Example LaTeX math expressions
latex_expressions = [
    r"\frac{a}{b}",
    r"\int_0^{\infty} e^{-x^2} dx",
    r"E = mc^2"
]

# Tokenize the LaTeX expressions
tokens = tokenizer(latex_expressions, padding=True, truncation=True, return_tensors="pt")

# Print tokenized input
print(tokens.input_ids)
 """

<>:1: SyntaxWarning: invalid escape sequence '\i'
<>:1: SyntaxWarning: invalid escape sequence '\i'
C:\Users\billl\AppData\Local\Temp\ipykernel_6740\354891771.py:1: SyntaxWarning: invalid escape sequence '\i'
  """ # test tokenize latex


' # test tokenize latex\nfrom transformers import AlbertTokenizer\nimport torch\n\n# Load the tokenizer\ntokenizer = AlbertTokenizer.from_pretrained(\'albert-base-v2\')\n\n# Example LaTeX math expressions\nlatex_expressions = [\n    r"\x0crac{a}{b}",\n    r"\\int_0^{\\infty} e^{-x^2} dx",\n    r"E = mc^2"\n]\n\n# Tokenize the LaTeX expressions\ntokens = tokenizer(latex_expressions, padding=True, truncation=True, return_tensors="pt")\n\n# Print tokenized input\nprint(tokens.input_ids)\n '

In [34]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Deep Learning library
import torch

# to load transformer models
#from sentence_transformers import SentenceTransformer
#from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline # T5 is Google

# to split the data
from sklearn.model_selection import train_test_split

# to compute performance metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score


In [35]:

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
files = []
# on vs code
data_dir = './eedi-mining/eedi-mining-misconceptions-in-mathematics'
miscon_file_index = 0
train_file_index = 3
test_file_index = 2
# on kaggle
# data_dir = '/kaggle/input'
#miscon_file_index = 1
#train_file_index = 2
#test_file_index = 3
for dirname, _, filenames in os.walk(data_dir):
    print (filenames)
    for filename in filenames:
        print(os.path.join(dirname, filename))
        files.append(os.path.join(dirname, filename))
misconceptions_filename = files[miscon_file_index]
train_filename = files[train_file_index]
test_filename = files[test_file_index]
# For Kaggle, You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

['misconception_mapping.csv', 'sample_submission.csv', 'test.csv', 'train.csv']
./eedi-mining/eedi-mining-misconceptions-in-mathematics\misconception_mapping.csv
./eedi-mining/eedi-mining-misconceptions-in-mathematics\sample_submission.csv
./eedi-mining/eedi-mining-misconceptions-in-mathematics\test.csv
./eedi-mining/eedi-mining-misconceptions-in-mathematics\train.csv


## File and Field Information
### [train/test].csv

* QuestionId - Unique question identifier (int).
* ConstructId - Unique construct identifier (int) .
* ConstructName - Most granular level of knowledge related to question (str).
* CorrectAnswer - A, B, C or D (char).
* SubjectId - Unique subject identifier (int).
* SubjectName - More general context than the construct (str).
* QuestionText - Question text extracted from the question image using human-in-the-loop OCR (str) .
* Answer[A/B/C/D]Text - Answer option A text extracted from the question image using human-in-the-loop OCR (str).
* Misconception[A/B/C/D]Id - Unique misconception identifier (int). Ground truth labels in train.csv; your task is to predict these labels for test.csv.

### misconception_mapping.csv
maps MisconceptionId to its MisconceptionName

### sample_submission.csv
A submission file in the correct format.
* QuestionId_Answer - Each question has three incorrect answers for which need you predict the MisconceptionId.
* MisconceptionId - You can predict up to 25 values, space delimited.

# get training data

In [36]:
train_data = pd.read_csv(train_filename)
train_data_orig = train_data.copy()
train_data.head()

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,NaN,NaN,NaN,1672.0
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,2142.0,143.0,2142.0,NaN
2,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,1287.0,NaN,1287.0,1073.0
3,3,2377,Recall and use the intersecting diagonals prop...,88,Properties of Quadrilaterals,C,The angles highlighted on this rectangle with ...,acute,obtuse,\( 90^{\circ} \),Not enough information,1180.0,1180.0,NaN,1180.0
4,4,3387,Substitute positive integer values into formul...,67,Substitution into Formula,A,The equation \( f=3 r^{2}+3 \) is used to find...,\( 30 \),\( 27 \),\( 51 \),\( 24 \),NaN,NaN,NaN,1818.0


In [37]:
print ("train_data shape")
print("rows",train_data.shape[0])
print("columns",train_data.shape[1])

train_data shape
rows 1869
columns 15


## Get test data

In [38]:
test_data = pd.read_csv(test_filename)
test_data_orig = test_data.copy()
test_data.head()
print ("test_data shape")
print("rows", test_data.shape[0])
print("columns",test_data.shape[1])

test_data shape
rows 3
columns 11


## Get misconception data


In [39]:
miscon_data = pd.read_csv(misconceptions_filename)
miscon_data.head()
print ("miscon_data shape")
print("rows", miscon_data.shape[0])
print("columns",miscon_data.shape[1])

miscon_data shape
rows 2587
columns 2


In [40]:
subject_groups = train_data.groupby(["SubjectId","SubjectName"]).count().sort_values('QuestionId',ascending=False).reset_index()
print(subject_groups.shape[0], " different subject groups")
subject_groups.head()

163  different subject groups


,SubjectId,SubjectName,QuestionId,ConstructId,ConstructName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId
0,64,Linear Equations,53,53,53,53,53,53,53,53,53,23,32,30,27
1,171,Linear Sequences (nth term),44,44,44,44,44,44,44,44,44,28,20,21,19
2,33,BIDMAS,37,37,37,37,37,37,37,37,37,20,21,23,24
3,65,Quadratic Equations,36,36,36,36,36,36,36,36,36,18,17,25,14
4,75,Area of Simple Shapes,36,36,36,36,36,36,36,36,36,22,26,23,24


In [41]:
print("percentages of null values")
pd.DataFrame({'Count':train_data.isnull().sum()[train_data.isnull().sum()>0],'Percentage':(train_data.isnull().sum()[train_data.isnull().sum()>0]/train_data.shape[0])*100})

percentages of null values


,Count,Percentage
MisconceptionAId,734,39.272338
MisconceptionBId,751,40.181915
MisconceptionCId,789,42.215088
MisconceptionDId,832,44.515784


## Clean up missing data

In [42]:
# records where more than one misconception id is missing
missing_a_misconception = train_data[(train_data.CorrectAnswer == "A") & (train_data.MisconceptionBId.isnull() | train_data.MisconceptionCId.isnull()  | train_data.MisconceptionDId.isnull()) ]
print(missing_a_misconception.shape)
missing_a_misconception.head()
train_data[(train_data.CorrectAnswer == "A")].shape

(244, 15)


(482, 15)

In [43]:
missing_b_misconpception = train_data[(train_data.CorrectAnswer == "B") & (train_data.MisconceptionAId.isnull() | train_data.MisconceptionCId.isnull()) ]
missing_b_misconpception.head()

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId
10,10,1414,Expand products of three binomials in the form...,335,Expanding Triple Brackets and more,B,John is expanding these three brackets:\n\(\n(...,\( +6 x^{2} \),\( \begin{array}{l}+6 x^{2} \\ -3 x^{2} \\ +5 ...,\( \begin{array}{l}+6 x^{2} \\ +3 x^{2} \\ +5 ...,\( \begin{array}{l}+6 x^{2} \\ -3 x^{2} \\ -5 ...,NaN,NaN,583.0,NaN
13,13,3065,Convert from hours to minutes,209,Time,B,Hannah's journey to camp is \( 3 \) hours and ...,\( 196 \) minutes,\( 18 \) minutes,\( 102 \) minutes,\( 12 \) minutes,NaN,NaN,161.0,NaN
18,18,261,Carry out missing number subtraction problems ...,211,Adding and Subtracting Negative Numbers,B,![Number line with -12 and -7 marked. Starting...,\( -7 \),\( -5 \),\( -2 \),\( -6 \),2179.0,NaN,NaN,1824.0
20,20,2306,"Given the volume of a cuboid, work out missing...",189,Volume of Prisms,B,The volume of this cuboid is \( 30 \mathrm{~cm...,\( 6 \mathrm{~cm} \),\( 5 \mathrm{~cm} \),\( 4 \mathrm{~cm} \),\( 25 \mathrm{~cm} \),NaN,NaN,NaN,1984.0
21,21,1434,Factorise a quadratic expression in the form a...,53,Factorising into a Double Bracket,B,Step 1: Factorise the following expression\n\n...,\( (3 x+2)(3 x+1) \),\( (3 x+2)(x+1) \),Cannot be factorised,\( (3 x+1)(x+2) \),2240.0,NaN,NaN,NaN


In [44]:
misc_con_data = pd.read_csv(misconceptions_filename)
misc_con_data.head()

,MisconceptionId,MisconceptionName
0,0,Does not know that angles in a triangle sum to...
1,1,Uses dividing fractions method for multiplying...
2,2,Believes there are 100 degrees in a full turn
3,3,Thinks a quadratic without a non variable term...
4,4,Believes addition of terms and powers of terms...


## Update data to prepare for creating vectors


In [45]:
# clean up question
# train_data['CleanQuestion'] = train_data['QuestionText'].replace('\n',' ',regex=True)

# create new data frame with just misconceptions
# Create a new empty DataFrame
miscon_model = pd.DataFrame()


In [46]:
# convert latext formatting to text
from pylatexenc.latex2text import LatexNodes2Text
import pylatexenc
# Convert LaTeX to plain text
# Function to convert LaTeX to text
def latex_to_text(latex_string):
    return LatexNodes2Text().latex_to_text(latex_string)

# repolace names with variables to get consistency on vectors
import spacy

# Load the spaCy model
# for running locally, need to do
# python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

# Function to replace all detected names with unique variables
def replace_names_with_variables(text):
    doc = nlp(text)
    name_counter = 1
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            var_name = f"NAME{name_counter}"
            text = text.replace(ent.text, var_name)
            name_counter += 1
    return text

#def clean_math_text(text):
    # clean_text = latex_to_text(text)
    # clean_text = replace_names_with_variables(clean_text)
    # remove new line and other characters
    # print('before',clean_text)
    #clean_text = clean_text.replace('\n',' ')
    # print('after', clean_text)
    
    #return clean_text 
import re

def clean_math_text(text):
    # Lowercase
    text = text.lower()
    # Remove special characters (keeping LaTeX symbols)
    text = re.sub(r'[^a-zA-Z0-9\s\\{}^_]', '', text)
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Example text
text = "Here's an example: \int_0^{\infty} e^{-x^2} dx.\n Let's clean it up!"

# Clean the text
cleaned_text = clean_math_text(text)
print(cleaned_text)


# remove special characters like new line
#miscon_model['subjectName'] = train_data.SubjectName
#miscon_model['constructName'] = train_data.ConstructName
#miscon_model['question'] = train_data['QuestionText'].apply(clean_math_text)
#  Creating the flattened DataFrame
import math
flattened_data = []
for index, row in train_data.iterrows():
    # put incorrect answer A into row
    # print ("a id",row.MisconceptionAId)

    if ((row.CorrectAnswer != "A") & (math.isnan(row.MisconceptionAId) == False)):
        misconception_text_row = miscon_data[(miscon_data.MisconceptionId==row.MisconceptionAId)]
        misconception_id = row.MisconceptionAId
        formatted_answer = clean_math_text(row['AnswerAText'])
        # flattened_data.append({'subjectName': row['SubjectName'], 'constructName': row['ConstructName'], 'latex_formatted_question': row['QuestionText'], 'question': clean_math_text(row['QuestionText']), 'latex_formatted_wrongAnswer': row['AnswerAText'], 'wrongAnswer': clean_math_text(row['AnswerAText']), 'misconception': misconception_row.MisconceptionName})
    if ((row.CorrectAnswer != "B") & (math.isnan(row.MisconceptionBId)==False)):
        misconception_text_row = miscon_data[(miscon_data.MisconceptionId==row.MisconceptionBId)]
        misconception_id = row.MisconceptionBId
        formatted_answer= clean_math_text(row['AnswerBText'])
        # flattened_data.append({'subjectName': row['SubjectName']}, 'constructName': row['ConstructName'], 'latex_formatted_question': row['QuestionText'], 'question': clean_math_text(row['QuestionText']), 'latext_formatted_wrongAnswer': row['AnswerBText'], 'wrongAnswer': clean_math_text(row['AnswerBText']), 'misconception': misconception_row.MisconceptionName})
    if ((row.CorrectAnswer != "C") & (math.isnan(row.MisconceptionCId) ==False)):
        misconception_text_row = miscon_data[(miscon_data.MisconceptionId==row.MisconceptionCId)]
        misconception_id = row.MisconceptionCId
        formatted_answer = clean_math_text(row['AnswerCText'])        
        # flattened_data.append({'subjectName': row['SubjectName'], 'constructName': row['ConstructName'], 'latex_formatted_question': row['QuestionText'], 'question': clean_math_text(row['QuestionText']), 'latext_formatted_wrongAnswer': row['AnswerCText'], 'wrongAnswer': clean_math_text(row['AnswerCText']), 'misconception': misconception_row.MisconceptionName})
    if ((row.CorrectAnswer != "D") & (math.isnan(row.MisconceptionDId) == False)):
        misconception_text_row = miscon_data[(miscon_data.MisconceptionId==row.MisconceptionDId)]
        misconception_id = row.MisconceptionDId
        formatted_answer =  clean_math_text(row['AnswerDText'])
        # flattened_data.append({'subjectName': row['SubjectName'], 'constructName': row['ConstructName'], 'latex_formatted_question': row['QuestionText'], 'question': clean_math_text(row['QuestionText']), 'latext_formatted_wrongAnswer': row['AnswerDText'], 'wrongAnswer': clean_math_text(row['AnswerDText']), 'misconception': misconception_row.MisconceptionName})

    flattened_data.append({'text': f"{clean_math_text(row['SubjectName'])} {clean_math_text(row['ConstructName'])} {clean_math_text(row['QuestionText'])} {formatted_answer}", "label": int(misconception_id)})

flattened_df = pd.DataFrame(flattened_data)
# Sort by the 'label' column in ascending order 
flattened_df = flattened_df.sort_values(by='label')

<>:49: SyntaxWarning: invalid escape sequence '\i'
<>:49: SyntaxWarning: invalid escape sequence '\i'
C:\Users\billl\AppData\Local\Temp\ipykernel_6740\2774360719.py:49: SyntaxWarning: invalid escape sequence '\i'
  text = "Here's an example: \int_0^{\infty} e^{-x^2} dx.\n Let's clean it up!"


heres an example \int_0^{\infty} e^{x^2} dx lets clean it up


In [47]:
flattened_df.head()

,text,label
1421,angles in triangles find missing angles in a s...,0
325,measuring angles identify the number of degree...,2
1057,factorising into a single bracket factorise a ...,3
998,simplifying expressions by collecting like ter...,4
902,simplifying expressions by collecting like ter...,4


In [48]:
# save flattened data
flattened_df.to_csv('./eedi-mining/model/flattened_misconceptions.csv', index=False)

In [49]:
# defining a function to compute the cosine similarity between two embedding vectors
def cosine_score(text):
    # encoding the text
    embeddings = model.encode(text)

    # calculating the L2 norm of the embedding vector
    norm1 = np.linalg.norm(embeddings[0])
    norm2 = np.linalg.norm(embeddings[1])

    # computing the cosine similarity
    cosine_similarity_score = ((np.dot(embeddings[0],embeddings[1]))/(norm1*norm2))

    return cosine_similarity_score

In [50]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader, random_split

# Sample text data and labels
# Prepare your dataset (replace with your own data)
texts = flattened_df['text'].to_list()
original_labels =flattened_df['label'].to_list()

# Create a mapping dictionary
label_mapping = {label: idx for idx, label in enumerate(original_labels)}

print(label_mapping)

# Original labels for the data
labels = torch.tensor(original_labels, dtype=torch.long)

# Map original labels to sequential labels
mapped_labels = torch.tensor([label_mapping[label.item()] for label in labels], dtype=torch.long)
print (mapped_labels)



{0: 0, 2: 1, 3: 2, 4: 4, 9: 5, 11: 10, 14: 11, 20: 12, 21: 13, 22: 14, 27: 15, 28: 16, 29: 17, 31: 22, 32: 23, 33: 24, 35: 25, 36: 26, 38: 27, 39: 29, 42: 31, 48: 33, 49: 34, 51: 35, 55: 38, 57: 40, 58: 41, 60: 42, 61: 46, 66: 47, 68: 49, 70: 50, 71: 54, 74: 56, 76: 57, 77: 65, 78: 66, 81: 68, 82: 76, 83: 77, 84: 78, 86: 79, 91: 81, 95: 82, 101: 83, 102: 84, 103: 85, 105: 89, 106: 90, 107: 93, 108: 99, 109: 100, 110: 105, 113: 113, 114: 114, 118: 115, 119: 116, 120: 121, 126: 123, 127: 124, 130: 125, 131: 128, 132: 129, 133: 131, 134: 134, 138: 135, 141: 136, 142: 137, 143: 139, 146: 141, 154: 142, 159: 145, 161: 151, 162: 152, 164: 155, 166: 157, 167: 160, 172: 166, 176: 167, 186: 170, 187: 171, 189: 172, 191: 174, 197: 175, 207: 176, 211: 177, 212: 179, 217: 184, 219: 189, 220: 196, 221: 197, 226: 200, 227: 201, 228: 202, 230: 203, 233: 204, 234: 206, 236: 207, 244: 208, 245: 209, 247: 210, 248: 211, 249: 212, 251: 213, 252: 214, 253: 215, 255: 221, 256: 225, 261: 226, 264: 227, 265:

In [51]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AlbertTokenizer


# Load the tokenizer and model
# Load model directly
from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("AnReu/math_pretrained_bert")
model = AutoModelForPreTraining.from_pretrained("AnReu/math_pretrained_bert")


class MathMisconceptionsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        print(labels)
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        # Tokenize the text
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # Flatten the encoding tensors
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['labels'] = torch.tensor(label, dtype=torch.long)
        logging.info(label)
        return item


# get data
model_dir = 'c:/ai_ml/kaggle/eedi-mining/model'
#           'C:\ai_ml\kaggle\eedi-mining\model\flattened_misconceptions.csv'
model_data=f"{model_dir}/flattened_misconceptions.csv"

out_dir = model_dir



from transformers import AlbertTokenizer

tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")
max_length = 256  # Maximum length for tokenization
#train_dataset = MathMisconceptionsDataset(train_texts, train_labels, tokenizer, max_length)
#val_dataset = MathMisconceptionsDataset(val_texts, val_labels, tokenizer, max_length)

# Create the dataset
dataset = MathMisconceptionsDataset(texts, mapped_labels, tokenizer, max_length)


# Split the dataset
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)

# Save the datasets
model_train_dataset_name=f"{model_dir}/math_misconceptions_train_dataset.pt"
torch.save(train_dataset, model_train_dataset_name)

model_val_dataset_name=f"{model_dir}/math_misconceptions_val_dataset.pt"
torch.save(val_dataset, model_val_dataset_name)



C:\Users\billl\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


tensor([   0,    1,    2,  ..., 1867, 1867, 1868])


In [52]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset,  batch_size=30, shuffle=True)
print(f" - Number of samples : {len(train_dataloader.dataset)} | Number of batches: {len(train_dataloader)}")

val_dataloader = DataLoader(val_dataset,  batch_size=30, shuffle=True)
print(f" - Number of samples : {len(val_dataloader.dataset)} | Number of batches: {len(val_dataloader)}")

for batch in val_dataloader:
    print(batch)

C:\Users\billl\AppData\Local\Temp\ipykernel_6740\82937343.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(label, dtype=torch.long)
2024-11-17 20:16:41 - INFO - tensor(450)
2024-11-17 20:16:41 - INFO - tensor(177)
2024-11-17 20:16:41 - INFO - tensor(1670)
2024-11-17 20:16:41 - INFO - tensor(400)
2024-11-17 20:16:41 - INFO - tensor(698)
2024-11-17 20:16:41 - INFO - tensor(79)
2024-11-17 20:16:41 - INFO - tensor(657)
2024-11-17 20:16:41 - INFO - tensor(971)
2024-11-17 20:16:41 - INFO - tensor(408)
2024-11-17 20:16:41 - INFO - tensor(1288)
2024-11-17 20:16:41 - INFO - tensor(496)
2024-11-17 20:16:41 - INFO - tensor(1560)
2024-11-17 20:16:41 - INFO - tensor(797)
2024-11-17 20:16:41 - INFO - tensor(885)
2024-11-17 20:16:41 - INFO - tensor(1341)
2024-11-17 20:16:41 - INFO - tensor(1168)
2024-11-17 20:16:41 - 

 - Number of samples : 1495 | Number of batches: 50
 - Number of samples : 374 | Number of batches: 13
{'input_ids': tensor([[    2,  2310,    17,  ...,     0,     0,     0],
        [    2,   683,   201,  ...,     0,     0,     0],
        [    2,    85,   231,  ...,     0,     0,     0],
        ...,
        [    2, 19583,   128,  ...,     0,     0,     0],
        [    2,  9138,  6907,  ...,     0,     0,     0],
        [    2, 22716,    13,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([ 450,  177, 1670,  400,  698

2024-11-17 20:16:41 - INFO - tensor(105)
2024-11-17 20:16:41 - INFO - tensor(820)
2024-11-17 20:16:41 - INFO - tensor(788)
2024-11-17 20:16:41 - INFO - tensor(89)
2024-11-17 20:16:41 - INFO - tensor(1276)
2024-11-17 20:16:41 - INFO - tensor(588)
2024-11-17 20:16:41 - INFO - tensor(674)
2024-11-17 20:16:41 - INFO - tensor(459)
2024-11-17 20:16:41 - INFO - tensor(50)
2024-11-17 20:16:41 - INFO - tensor(786)
2024-11-17 20:16:41 - INFO - tensor(783)
2024-11-17 20:16:41 - INFO - tensor(1006)
2024-11-17 20:16:41 - INFO - tensor(400)
2024-11-17 20:16:41 - INFO - tensor(1079)
2024-11-17 20:16:41 - INFO - tensor(501)
2024-11-17 20:16:41 - INFO - tensor(1843)
2024-11-17 20:16:41 - INFO - tensor(1414)
2024-11-17 20:16:41 - INFO - tensor(510)
2024-11-17 20:16:41 - INFO - tensor(377)
2024-11-17 20:16:42 - INFO - tensor(1629)
2024-11-17 20:16:42 - INFO - tensor(76)
2024-11-17 20:16:42 - INFO - tensor(413)
2024-11-17 20:16:42 - INFO - tensor(1055)
2024-11-17 20:16:42 - INFO - tensor(1367)
2024-11-17 

{'input_ids': tensor([[    2, 15154, 11689,  ...,     0,     0,     0],
        [    2, 18542, 10885,  ...,     0,     0,     0],
        [    2, 11173,  9539,  ...,     0,     0,     0],
        ...,
        [    2, 10095,    17,  ...,     0,     0,     0],
        [    2, 19910,    77,  ...,     0,     0,     0],
        [    2,  6745, 11173,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([ 105,  820,  788,   89, 1276,  588,  674,  459,   50,  786,  783, 1006,
         400, 1079,  501, 1843, 1414,  510,  377, 1629,   7

2024-11-17 20:16:42 - INFO - tensor(344)
2024-11-17 20:16:42 - INFO - tensor(284)
2024-11-17 20:16:42 - INFO - tensor(1094)
2024-11-17 20:16:42 - INFO - tensor(1418)
2024-11-17 20:16:42 - INFO - tensor(1203)
2024-11-17 20:16:42 - INFO - tensor(1372)
2024-11-17 20:16:42 - INFO - tensor(82)
2024-11-17 20:16:42 - INFO - tensor(1276)
2024-11-17 20:16:42 - INFO - tensor(730)
2024-11-17 20:16:42 - INFO - tensor(1753)
2024-11-17 20:16:42 - INFO - tensor(1507)
2024-11-17 20:16:42 - INFO - tensor(533)
2024-11-17 20:16:42 - INFO - tensor(1079)
2024-11-17 20:16:42 - INFO - tensor(608)
2024-11-17 20:16:42 - INFO - tensor(1397)
2024-11-17 20:16:42 - INFO - tensor(1283)
2024-11-17 20:16:42 - INFO - tensor(924)
2024-11-17 20:16:42 - INFO - tensor(155)
2024-11-17 20:16:42 - INFO - tensor(545)
2024-11-17 20:16:42 - INFO - tensor(196)
2024-11-17 20:16:42 - INFO - tensor(1576)
2024-11-17 20:16:42 - INFO - tensor(139)
2024-11-17 20:16:42 - INFO - tensor(257)
2024-11-17 20:16:42 - INFO - tensor(750)
2024-1

...

{'input_ids': tensor([[    2,  6745, 11173,  ...,     0,     0,     0],
        [    2,  6745, 12853,  ...,     0,     0,     0],
        [    2,   560,    68,  ...,     0,     0,     0],
        ...,
        [    2,    89, 11173,  ...,     0,     0,     0],
        [    2,  4721,    17,  ...,     0,     0,     0],
        [    2, 26314,    68,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([ 152,  885,  907, 1753,  468, 1025,  440, 1789,  675,  764, 1578, 1437,
         835,  341, 1131,  206,  744,  914, 1310, 1298,  91

2024-11-17 20:16:43 - INFO - tensor(581)
2024-11-17 20:16:43 - INFO - tensor(1181)
2024-11-17 20:16:43 - INFO - tensor(379)
2024-11-17 20:16:43 - INFO - tensor(841)
2024-11-17 20:16:43 - INFO - tensor(777)
2024-11-17 20:16:43 - INFO - tensor(1428)
2024-11-17 20:16:43 - INFO - tensor(815)
2024-11-17 20:16:43 - INFO - tensor(1349)
2024-11-17 20:16:43 - INFO - tensor(1080)
2024-11-17 20:16:43 - INFO - tensor(113)
2024-11-17 20:16:43 - INFO - tensor(128)
2024-11-17 20:16:43 - INFO - tensor(1845)
2024-11-17 20:16:43 - INFO - tensor(1236)
2024-11-17 20:16:43 - INFO - tensor(1326)
2024-11-17 20:16:43 - INFO - tensor(818)
2024-11-17 20:16:43 - INFO - tensor(1849)
2024-11-17 20:16:43 - INFO - tensor(1124)
2024-11-17 20:16:43 - INFO - tensor(909)
2024-11-17 20:16:43 - INFO - tensor(786)
2024-11-17 20:16:43 - INFO - tensor(916)
2024-11-17 20:16:43 - INFO - tensor(170)
2024-11-17 20:16:43 - INFO - tensor(1569)
2024-11-17 20:16:43 - INFO - tensor(1642)
2024-11-17 20:16:43 - INFO - tensor(783)
2024-

{'input_ids': tensor([[    2,  6960,  8676,  ...,     0,     0,     0],
        [    2,  5334, 10459,  ...,     0,     0,     0],
        [    2,   683,   201,  ...,     0,     0,     0],
        ...,
        [    2,  6089,   217,  ...,     0,     0,     0],
        [    2, 28257,    68,  ...,     0,     0,     0],
        [    2,  3007,    14,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([ 597,  207, 1779, 1389,   77,  487,  362,  715, 1236,  808, 1211, 1501,
        1594, 1366, 1589, 1389, 1155, 1134, 1461, 1478,  26

2024-11-17 20:16:43 - INFO - tensor(1159)
2024-11-17 20:16:43 - INFO - tensor(1035)
2024-11-17 20:16:43 - INFO - tensor(1812)
2024-11-17 20:16:43 - INFO - tensor(459)
2024-11-17 20:16:43 - INFO - tensor(888)
2024-11-17 20:16:43 - INFO - tensor(145)
2024-11-17 20:16:43 - INFO - tensor(1764)
2024-11-17 20:16:43 - INFO - tensor(463)
2024-11-17 20:16:43 - INFO - tensor(1709)
2024-11-17 20:16:43 - INFO - tensor(1789)
2024-11-17 20:16:43 - INFO - tensor(1389)
2024-11-17 20:16:43 - INFO - tensor(474)
2024-11-17 20:16:43 - INFO - tensor(1858)
2024-11-17 20:16:43 - INFO - tensor(493)
2024-11-17 20:16:43 - INFO - tensor(1611)
2024-11-17 20:16:43 - INFO - tensor(1006)
2024-11-17 20:16:43 - INFO - tensor(1233)
2024-11-17 20:16:43 - INFO - tensor(1032)
2024-11-17 20:16:43 - INFO - tensor(1773)
2024-11-17 20:16:43 - INFO - tensor(513)
2024-11-17 20:16:43 - INFO - tensor(1112)
2024-11-17 20:16:43 - INFO - tensor(536)
2024-11-17 20:16:43 - INFO - tensor(1693)
2024-11-17 20:16:43 - INFO - tensor(907)
2

{'input_ids': tensor([[    2,  5780,    18,  ...,     0,     0,     0],
        [    2,  3704,    16,  ...,     0,     0,     0],
        [    2, 10095,    17,  ...,     0,     0,     0],
        ...,
        [    2,  9138,  6907,  ...,     0,     0,     0],
        [    2,   293, 17191,  ...,     0,     0,     0],
        [    2, 19910,    77,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1551,  986, 1159, 1035, 1812,  459,  888,  145, 1764,  463, 1709, 1789,
        1389,  474, 1858,  493, 1611, 1006, 1233, 1032, 177

In [53]:
from transformers import AlbertForSequenceClassification, TrainingArguments, Trainer

# Load the model
model = AlbertForSequenceClassification.from_pretrained("AnReu/math_albert", num_labels=2)


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at AnReu/math_albert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [56]:
from transformers import AdamW, get_scheduler
from torch.optim import AdamW
from tqdm.auto import tqdm

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Learning rate scheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

# Training loop
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}")
    for batch in progress_bar:
        batch = {k: v.to(device) for k, v in batch.items()}
        print(batch)
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.set_postfix({"loss": loss.item()})

    # Validation loop (optional)
    model.eval()
    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            print(batch)
            outputs = model(**batch)
            val_loss = outputs.loss
            print(f"Validation loss: {val_loss.item()}")


Epoch 1/3:   0%|          | 0/748 [00:00<?, ?it/s]

C:\Users\billl\AppData\Local\Temp\ipykernel_6740\82937343.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(label, dtype=torch.long)
2024-11-17 20:22:13 - INFO - tensor(400)
2024-11-17 20:22:13 - INFO - tensor(1693)


{'input_ids': tensor([[    2,  4721,    17, 27031,    68,    29, 26380,    18,  3547, 26380,
            18,   113,    14,  2116,  1013,    57,    21,   421,   234,    16,
         26380,  1489,   184,   378,    42,   293,    71,    14, 19076,    20,
         18469,    13,     1,   713,  3318,  3963,    13,     1,    21,  4698,
           848,    16,   713,  2984,   713,  3963,    29,    14, 26380,   819,
          9011,    71,     3,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

IndexError: Target 400 is out of bounds.

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch"
)

In [ ]:
# Initialize the Trainer
logging.info("initialize trainer")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

# Train the model
logging.info('training the model')
trainer.train()

logging.info('evaluate model')
results = trainer.evaluate()
print(results)
